- OnBase
    - rm_DVStatementRequests [x]
    - workitemlc
    - rmobject
    - rm_DVJobs
    - lcstate
    - rm_DVStatementRequestActivityRecords
    - rm_DVStatements [x]
    - rm_DVProjects
- BSAP
    - SNotes

In [0]:
from pyspark.sql import DataFrame
from pyspark.sql.readwriter import DataFrameReader
from pyspark.sql.functions import col, lit, trim, when
from typing import Dict, List

def databaseReader(readFormat: str, readOptions: Dict[str, str]) -> DataFrameReader:
    return (
        spark
        .read
        .format(readFormat)
        .options(**readOptions)
    )
    

def truncateStrings(dataFrame: DataFrame) -> DataFrame:
    for colName, dtype in dataFrame.dtypes:
        if dtype == 'string':
            column = trim(colName)
            case = when(column == '', lit(None).cast('string')).otherwise(column)
            dataFrame = dataFrame.withColumn(colName, case)

    return dataFrame

In [0]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import min, max
from typing import Dict, Optional

def partitionColumn(
    dataFrame: DataFrame,
    column: str,
    partitions: int=sc.defaultParallelism
) -> Optional[Dict[str, str]]:
    bounds = (
        dataFrame
        .select(
            min(column).cast('string'),
            max(column).cast('string')
        )
        .na
        .drop(how='any')
        .first()
    )

    if bounds:
        return {
            'partitionColumn': column,
            'lowerBound': bounds[0],
            'upperBound': bounds[1],
            'numPartitions': partitions
        }
    else:
        None

In [0]:
import os

PATH = '/tmp/requests'
BRONZE = os.path.join(PATH, 'bronze')
SILVER = os.path.join(PATH, 'silver')

options = {
    'url': 'jdbc:sqlserver://',
    'server': '10.0.2.16',
    'database': 'OnBase',
    'user': dbutils.secrets.get('silo-key-vault-scope', 'databricks-base-user'),
    'password': dbutils.secrets.get('silo-key-vault-scope', 'databricks-base-password')
}

reader = databaseReader('com.microsoft.sqlserver.jdbc.spark', options)

In [0]:
query = """
(SELECT
    [Reference Number] as ReferenceNumber,
    -- JobNo,
    -- JobTier,
    [Statement Date] as StatementDate,
    -- [Document Handle],
    ObjectID,
    -- CustVendorObjID,
    -- CustVendGroupObjID,
    -- CustVendGroupName,
    -- CustomerName,
    -- CID,
    -- CustVendName,
    -- CustVendNo,
    -- Volume,
    -- [Accounts Identified],
    -- Recon,
    EmailMessageID,
    CreatedDate,
    SRARObjectId
    -- ZeroBalance
FROM
    hsi.rm_DVStatements) as query
"""

table = (
    reader
    .option('dbtable', query)
    .load()
)

table = truncateStrings(table)

path = os.path.join(BRONZE, 'Statements')

(
    table
    .write
    .mode('overwrite')
    .parquet(path)
)

display(spark.read.parquet(path))

ReferenceNumber,StatementDate,ObjectID,EmailMessageID,CreatedDate,SRARObjectId
1224005,2019-08-14T00:00:00.000+0000,20513258,00000000F643DA057DBD124B829A30CFCF400C1507002B871AC579DA7041B55C292E8DC8BBBA000000CBE8AA0000AC80609CDFFC214CBE54978FCA484FD6000231DA71920000,2019-08-14T11:18:01.353+0000,null
1229524,2019-08-14T00:00:00.000+0000,20513265,00000000F643DA057DBD124B829A30CFCF400C1507002B871AC579DA7041B55C292E8DC8BBBA000000CBE8AA0000AC80609CDFFC214CBE54978FCA484FD6000231DC72A50000,2019-08-14T11:18:26.637+0000,null
1247505,2019-08-14T00:00:00.000+0000,20513268,00000000F643DA057DBD124B829A30CFCF400C1507002B871AC579DA7041B55C292E8DC8BBBA000000CBE8AA0000AC80609CDFFC214CBE54978FCA484FD6000231DC72790000,2019-08-14T11:19:08.073+0000,null
1248087,2019-08-14T00:00:00.000+0000,20513273,00000000F643DA057DBD124B829A30CFCF400C1507002B871AC579DA7041B55C292E8DC8BBBA000000CBE8AA0000AC80609CDFFC214CBE54978FCA484FD6000231DC72780000,2019-08-14T11:19:43.393+0000,null
1224005,2019-08-14T00:00:00.000+0000,20513254,00000000F643DA057DBD124B829A30CFCF400C1507002B871AC579DA7041B55C292E8DC8BBBA000000CBE8AA0000AC80609CDFFC214CBE54978FCA484FD6000231DA71920000,2019-08-14T11:17:48.653+0000,null
1223925,2019-08-14T00:00:00.000+0000,20513249,00000000F643DA057DBD124B829A30CFCF400C1507002B871AC579DA7041B55C292E8DC8BBBA000000CBE8AA0000AC80609CDFFC214CBE54978FCA484FD6000231DC72AB0000,2019-08-14T11:17:15.840+0000,null
1223925,2019-08-14T00:00:00.000+0000,20513238,00000000F643DA057DBD124B829A30CFCF400C1507002B871AC579DA7041B55C292E8DC8BBBA000000CBE8AA0000AC80609CDFFC214CBE54978FCA484FD6000231DC72AB0000,2019-08-14T11:16:58.657+0000,null
1223925,2019-08-14T00:00:00.000+0000,20513233,00000000F643DA057DBD124B829A30CFCF400C1507002B871AC579DA7041B55C292E8DC8BBBA000000CBE8AA0000AC80609CDFFC214CBE54978FCA484FD6000231DC72AB0000,2019-08-14T11:16:44.880+0000,null
1223925,2019-08-14T00:00:00.000+0000,20513228,00000000F643DA057DBD124B829A30CFCF400C1507002B871AC579DA7041B55C292E8DC8BBBA000000CBE8AA0000AC80609CDFFC214CBE54978FCA484FD6000231DC72AB0000,2019-08-14T11:16:26.217+0000,null
1244129,2019-08-14T00:00:00.000+0000,20512590,00000000F643DA057DBD124B829A30CFCF400C1507002B871AC579DA7041B55C292E8DC8BBBA000000CBE8AA0000AC80609CDFFC214CBE54978FCA484FD6000231DA717D0000,2019-08-14T09:40:30.830+0000,null


In [0]:
query = """
(SELECT
    -- JobNo,
    -- JobTier,
    -- CustomerName,
    -- CustVendorID,
    -- VendorNo,
    -- CustVendorGroupID,
    WNC,
    -- StatementWNC,
    -- VendorGroupName,
    -- Volume,
    -- VolumeTier,
    -- VolumeLast12,
    RequestDate,
    -- ReferenceNumber,
    Status,
    RequestMethod,
    RequestType,
    Contact,
    RequesterFullName,
    -- RequestText,
    LastActivityDate,
    LastStatementReceivedDate,
    -- CreatedDate,
    -- CallsheetNo,
    ObjectID
    -- CallerStatus,
    -- ReconStatus,
    -- CurrentAssigneeID,
    -- CurrentAssigneeName,
    -- EnteredReconDate,
    -- LastReconQueueName,
    -- LastReconQueueEntryDate,
    -- AccountsReceived,
    -- AccountsRequested,
    -- MessageID,
    -- VendorContactObjectID,
    -- WebsiteVendor,
    -- WNCSpecialHandling,
    -- NeedLeadVendor,
    -- VendorGroupPrimaryAccountType
FROM
    hsi.rm_DVStatementRequests) as query
"""

table = (
    reader
    .option('dbtable', query)
    .load()
)

table = truncateStrings(table)

path = os.path.join(BRONZE, 'StatementRequests')

(
    table
    .write
    .mode('overwrite')
    .parquet(path)
)

display(spark.read.parquet(path))

WNC,RequestDate,Status,RequestMethod,RequestType,Contact,RequesterFullName,LastActivityDate,LastStatementReceivedDate,ObjectID
0,2020-05-01T00:00:00.000+0000,Fully Received,MassEmail,null,ar@system1us.com,Jessica Rinehart,2020-05-01T00:00:00.000+0000,2020-05-04T00:00:00.000+0000,25422373
null,2020-05-01T00:00:00.000+0000,Fully Received,MassEmail,null,courtney.groden@omadahealth.com,Jessica Rinehart,2020-05-01T00:00:00.000+0000,null,25422392
null,2020-05-01T00:00:00.000+0000,Fully Received,MassEmail,null,ar@adreima.com,Jessica Rinehart,2020-05-01T00:00:00.000+0000,2020-05-04T00:00:00.000+0000,25422433
null,2020-05-01T00:00:00.000+0000,Fully Received,MassEmail,null,rebecca@dashcourier.com,Jessica Rinehart,2020-05-01T00:00:00.000+0000,2020-05-04T00:00:00.000+0000,25422445
null,2020-05-01T00:00:00.000+0000,Partial Receipt,MassEmail,null,CBaker@cbccts.org,Jessica Rinehart,2020-05-01T00:00:00.000+0000,2020-05-04T00:00:00.000+0000,25422454
0,2020-05-01T00:00:00.000+0000,Fully Received,MassEmail,null,lgilmore@mcomc.com,Jessica Rinehart,2020-05-01T00:00:00.000+0000,2020-05-01T00:00:00.000+0000,25422500
0,2020-05-01T00:00:00.000+0000,Fully Received,MassEmail,null,Accounts.Receivable@ULINE.COM,Jessica Rinehart,2020-11-02T00:00:00.000+0000,null,25422522
0,2020-05-01T00:00:00.000+0000,Fully Received,MassEmail,null,accountsreceivable@heartflow.com,Jessica Rinehart,2020-05-01T00:00:00.000+0000,null,25422526
null,2020-05-01T00:00:00.000+0000,Fully Received,MassEmail,null,US.AR@BRAINLAB.COM,Jessica Rinehart,2020-05-01T00:00:00.000+0000,null,25422532
0,2020-05-01T00:00:00.000+0000,Fully Received,MassEmail,null,ar@biofiredx.com,Jessica Rinehart,2020-05-01T00:00:00.000+0000,null,25422539


In [0]:
query = """
(SELECT
    ObjectID,
    CreatedDate,
    ReferenceNumber,
    CustomerVendorName,
    JobNumber,
    JobName,
    ContactType,
    ActivityUser,
    -- ActivityDate,
    -- FollowUpDate,
    -- STNID,
    -- Notes,
    Outcome,
    ActivityType,
    -- CustVendorObjectID,
    VendorContactObjectID,
    StatementRequestObjectID
    -- CreatedBy,
    -- MessageID
FROM
    hsi.rm_DVStatementRequestActivityRecords) as query
"""

table = (
    reader
    .option('dbtable', query)
    .load()
)

table = truncateStrings(table)

path = os.path.join(BRONZE, 'StatementRequestActivityRecords')

(
    table
    .write
    .mode('overwrite')
    .parquet(path)
)

display(spark.read.parquet(path).limit(5))

ObjectID,CreatedDate,ReferenceNumber,CustomerVendorName,JobNumber,JobName,ContactType,ActivityUser,Outcome,ActivityType,VendorContactObjectID,StatementRequestObjectID
28785475,2020-11-02T11:41:33.170+0000,1279250,ULINE,3412,Carolinas Shared Service - 3412,Call,ASIBLEY,null,null,null,25422522
28785637,2020-11-02T11:47:27.003+0000,1279250,ULINE,3412,Carolinas Shared Service - 3412,Email,ASIBLEY,null,null,null,25422522
28786094,2020-11-02T11:52:58.307+0000,1279250,ULINE,3412,Carolinas Shared Service - 3412,Client Email,ASIBLEY,null,null,null,25422522
28838136,2020-11-04T17:47:58.533+0000,1287154,BIOCOMPOSITES INC.,3435,The Christ Hospital - 3435,null,bwilliams1,null,null,null,27781594
28838138,2020-11-04T17:48:02.617+0000,1225130,BONA FIDE COMMERCIAL SERVICES,3177,UC Health - 3177,null,jdagher,null,null,null,25860124


In [0]:
query = """
(SELECT
    lcnum,
    statenum,
    contentnum,
    wfcontenttype,
    transdate,
    priority,
    versionid,
    ownernum,
    ownedstatus,
    ownedsince,
    lastupdated,
    flags,
    contentclassnum
FROM
    hsi.workitemlc) as query
"""

table = (
    reader
    .option('dbtable', query)
    .load()
)

table = truncateStrings(table)

path = os.path.join(BRONZE, 'WorkItems')

(
    table
    .write
    .mode('overwrite')
    .parquet(path)
)

display(spark.read.parquet(path))

lcnum,statenum,contentnum,wfcontenttype,transdate,priority,versionid,ownernum,ownedstatus,ownedsince,lastupdated,flags,contentclassnum
125,298,11190595,3,2018-02-11T07:07:08.553+0000,0,0,0,0,1964-01-01T00:00:00.000+0000,1964-01-01T00:00:00.000+0000,0,1184
125,298,11190596,3,2018-02-11T07:07:08.630+0000,0,0,0,0,1964-01-01T00:00:00.000+0000,1964-01-01T00:00:00.000+0000,0,1184
125,298,11190597,3,2018-02-11T07:07:08.710+0000,0,0,0,0,1964-01-01T00:00:00.000+0000,1964-01-01T00:00:00.000+0000,0,1184
125,298,11190598,3,2018-02-11T07:07:08.787+0000,0,0,0,0,1964-01-01T00:00:00.000+0000,1964-01-01T00:00:00.000+0000,0,1184
125,298,11190599,3,2018-02-11T07:07:08.880+0000,0,0,0,0,1964-01-01T00:00:00.000+0000,1964-01-01T00:00:00.000+0000,0,1184
125,298,11190600,3,2018-02-11T07:07:08.960+0000,0,0,0,0,1964-01-01T00:00:00.000+0000,1964-01-01T00:00:00.000+0000,0,1184
125,298,11190601,3,2018-02-11T07:07:09.037+0000,0,0,0,0,1964-01-01T00:00:00.000+0000,1964-01-01T00:00:00.000+0000,0,1184
125,298,11190602,3,2018-02-11T07:07:09.117+0000,0,0,0,0,1964-01-01T00:00:00.000+0000,1964-01-01T00:00:00.000+0000,0,1184
125,298,11190603,3,2018-02-11T07:07:09.240+0000,0,0,0,0,1964-01-01T00:00:00.000+0000,1964-01-01T00:00:00.000+0000,0,1184
125,298,11190604,3,2018-02-11T07:07:09.380+0000,0,0,0,0,1964-01-01T00:00:00.000+0000,1964-01-01T00:00:00.000+0000,0,1184


In [0]:
query = """
(SELECT
    objectid,
    rmobjectname,
    parentobjectid,
    classid,
    rmcreatedby,
    createddate,
    writestatus,
    statusid,
    activestatus
FROM
    hsi.rmobject) as query
"""

# table = reader.option('dbtable', query).load()
# partitionMap = partitionColumn(table, 'objectid')
# partitionOptions = {**options, **partitionMap}

table = (
    spark
    .read
    .format('com.microsoft.sqlserver.jdbc.spark')
    .options(**partitionOptions)
    .option('dbtable', 'hsi.rmobject')
    .load()
)

table = truncateStrings(table)

path = os.path.join(BRONZE, 'Objects')

(
    table
    .write
    .mode('overwrite')
    .parquet(path)
)

display(spark.read.parquet(path))

objectid,rmobjectname,parentobjectid,classid,rmcreatedby,createddate,writestatus,statusid,activestatus
193512,null,0,1094,MANAGER,2016-02-29T14:11:27.440+0000,0,0,1
193513,null,0,1094,ASMITTER,2016-03-01T07:58:00.210+0000,0,0,0
193514,null,0,1093,ASMITTER,2016-03-01T08:01:14.907+0000,0,0,0
193515,null,0,1093,ASMITTER,2016-03-01T08:02:20.740+0000,0,0,0
193516,null,0,1093,ASMITTER,2016-03-01T08:03:01.927+0000,0,0,0
193517,null,0,1093,ASMITTER,2016-03-01T08:04:04.500+0000,0,0,0
193518,null,0,1093,ASMITTER,2016-03-01T08:04:41.910+0000,0,0,0
193519,null,0,1093,ASMITTER,2016-03-01T08:05:10.243+0000,0,0,0
193520,null,0,1093,ASMITTER,2016-03-01T08:05:46.513+0000,0,0,0
193521,null,0,1093,ASMITTER,2016-03-01T08:06:25.483+0000,0,0,0


In [0]:
query = """
(SELECT
    JobNo,
    ManagerID,
    SupervisorID,
    SummaryInstructions,
    AuditPeriodBase,
    PTID,
    PortalInvoiceEntryGoal,
    PortalCreditEntryGoal,
    PortalDaysToPayGoal,
    EmailAccess,
    Active,
    ImagingAccess,
    ManagerPodName,
    Name,
    DisplayAuditPeriod,
    VisibleInPortal,
    CustomerID,
    Application,
    CID,
    Rolling,
    [Customer Customer SILO Group Group Code] as CustomerGroupCode,
    PortalFriendlyName,
    PortalClaimsReadOnly,
    CustomerGroupID,
    ManagerPodID,
    SortOrder,
    StatementCallsheetAssignee,
    [Customer Customer SILO Group Name] as CustomerGroupName,
    [Service Line] as ServiceLine,
    [Service Type] as ServiceType
FROM
    hsi.rm_DVJobs) as query
"""

# query = 'hsi.rm_DVJobs'

table = (
    reader
    .option('dbtable', query)
    .load()
)

table = truncateStrings(table)

path = os.path.join(BRONZE, 'Jobs')

(
    table
    .write
    .mode('overwrite')
    .parquet(path)
)

display(spark.read.parquet(path))

JobNo,ManagerID,SupervisorID,SummaryInstructions,AuditPeriodBase,PTID,PortalInvoiceEntryGoal,PortalCreditEntryGoal,PortalDaysToPayGoal,EmailAccess,Active,ImagingAccess,ManagerPodName,Name,DisplayAuditPeriod,VisibleInPortal,CustomerID,Application,CID,Rolling,CustomerGroupCode,PortalFriendlyName,PortalClaimsReadOnly,CustomerGroupID,ManagerPodID,SortOrder,StatementCallsheetAssignee,CustomerGroupName,ServiceLine,ServiceType
2600,null,null,null,null,19080042,null,null,null,null,0,null,null,"BSI Healthcare Audit Services, LLC - 2600",12/30/1899,1,19080038,Review With Supervisor,15121,0,null,null,1,19080041,null,null,null,null,Other,Programming
3570,null,null,null,null,33494516,null,null,null,null,1,Unknown,null,Weiser Memorial Hospital - 3570,null,0,33494513,Review With Supervisor,15722,0,null,null,1,33494514,null,0,null,null,Pharmacy Services,Rx - Audit
3178,null,null,null,null,33493822,null,null,null,null,1,Unknown,null,Jung Family Trust - 3178,null,0,33493819,Review With Supervisor,15438,0,null,null,1,33493820,null,0,null,null,Profit Recovery,Sales & Use Tax
3447,null,null,null,null,33493844,null,null,null,null,1,Unknown,null,Northern Cochise Community Hospital Inc - 3447,null,0,33493840,Review With Supervisor,15601,0,null,null,1,33493841,null,0,null,null,Pharmacy Services,Rx - Audit
3467,null,null,null,null,33493944,null,null,null,null,1,Unknown,null,Lake Chelan Community Hospital & Clinic - 3467,null,0,33493941,Review With Supervisor,15618,0,null,null,1,33493942,null,0,null,null,Pharmacy Services,Rx - Audit
3477,null,null,null,null,33494003,null,null,null,null,1,Unknown,null,Fairbanks Memorial Hospital - 3477,null,0,33494000,Review With Supervisor,15628,0,null,null,1,33494001,null,0,null,null,Pharmacy Services,Rx - Audit
3478,null,null,null,null,33494007,null,null,null,null,1,Unknown,null,Faxton - 3478,null,0,33494004,Review With Supervisor,15629,0,null,null,1,33494005,null,0,null,null,Pharmacy Services,Rx - Audit
3479,null,null,null,null,33494012,null,null,null,null,1,Unknown,null,Greater Philly Health - 3479,null,0,33494009,Review With Supervisor,15630,0,null,null,1,33494010,null,0,null,null,Pharmacy Services,Rx - Audit
3480,null,null,null,null,33494017,null,null,null,null,1,Unknown,null,Harrison Community Hospital - 3480,null,0,33494013,Review With Supervisor,15631,0,null,null,1,33494014,null,0,null,null,Pharmacy Services,Rx - Audit
3485,null,null,null,null,33494049,null,null,null,null,1,Unknown,null,Kaweah Delta Health Care District - 3485,null,0,33494042,Review With Supervisor,15636,0,null,null,1,33494044,null,0,null,null,Pharmacy Services,Rx - Audit


In [0]:
query = """
(SELECT
    statenum,
    statename,
    statedesc,
    statehelp,
    validactionflags,
    bitmapnum,
    iconnum,
    queuetype,
    defsystemwork,
    defuserwork,
    scope,
    flags,
    foldertypenum,
    templatenum,
    docseltasklistnum,
    cqnum,
    loadbalancingwork,
    refresh,
    requiredrole,
    flags2,
    keynum,
    obrefresh,
    wftimerservername,
    configversion
FROM
    hsi.lcstate) as query
"""

table = (
    reader
    .option('dbtable', query)
    .load()
)

table = truncateStrings(table)

path = os.path.join(BRONZE, 'States')

(
    table
    .write
    .mode('overwrite')
    .parquet(path)
)

display(spark.read.parquet(path))

statenum,statename,statedesc,statehelp,validactionflags,bitmapnum,iconnum,queuetype,defsystemwork,defuserwork,scope,flags,foldertypenum,templatenum,docseltasklistnum,cqnum,loadbalancingwork,refresh,requiredrole,flags2,keynum,obrefresh,wftimerservername,configversion
185,SYS - Initial,null,null,0,2305420,2305110,0,678,679,120,0,0,0,0,0,0,null,0,0,0,0,BSIHC-GVL-APP01,18000069
186,SYS - Done,null,null,0,2305420,2305110,0,680,681,120,0,0,0,0,0,0,null,0,0,0,0,null,18000069
187,Claim Image Errors,null,null,0,2305591,2305298,0,682,683,120,0,0,0,0,0,0,null,0,1,0,0,null,18000069
188,SYS - Initial,null,null,0,2305420,2305110,0,690,691,121,0,0,0,0,0,0,null,0,0,0,0,BSIHC-GVL-APP01,18000069
189,Assignment,null,null,0,2305420,2305110,0,701,702,121,4,0,0,0,147,0,null,0,0,0,0,BSIHC-GVL-APP01,18000069
190,Working Callsheets,null,null,0,2305420,2305110,8192,708,709,121,0,0,0,0,0,0,null,0,0,0,0,BSIHC-GVL-APP01,18000069
191,SYS - Done,null,null,0,2305420,2305110,0,711,712,121,0,0,0,0,0,0,null,0,0,0,0,null,18000069
182,SYS - Initial [USA],null,null,0,0,0,0,665,666,119,0,0,0,0,0,0,null,0,0,0,0,null,null
183,SYS - Update Statement Autofill,null,null,0,0,0,0,667,668,119,0,0,0,0,0,0,null,0,0,0,0,null,null
184,SYS - Done [USA],null,null,0,0,0,0,669,670,119,0,0,0,0,0,0,null,0,0,0,0,null,null


In [0]:
query = """
(SELECT
    ProjectNo,
    Name,
    JobNo,
    Status,
    PrimaryAuditorID,
    SecondaryAuditorID,
    StartDate,
    EndDate,
    ProjectType,
    VendorPotentialThreshold,
    WorkingItemThreshold,
    ImagingAccess,
    PercentComplete,
    AvailableInPortal,
    PortalFriendlyName,
    RollingOOSInterval,
    UsesModernWorkingItems,
    AgedOpenCreditDate,
    AgedOpenCreditMinimum,
    DebitsProject,
    KillOnClosed,
    ServiceType,
    ServiceLine
FROM
    hsi.rm_DVProjects) as query
"""

table = (
    reader
    .option('dbtable', query)
    .load()
)

table = truncateStrings(table)

path = os.path.join(BRONZE, 'Projects')

(
    table
    .write
    .mode('overwrite')
    .parquet(path)
)

display(spark.read.parquet(path))

ProjectNo,Name,JobNo,Status,PrimaryAuditorID,SecondaryAuditorID,StartDate,EndDate,ProjectType,VendorPotentialThreshold,WorkingItemThreshold,ImagingAccess,PercentComplete,AvailableInPortal,PortalFriendlyName,RollingOOSInterval,UsesModernWorkingItems,AgedOpenCreditDate,AgedOpenCreditMinimum,DebitsProject,KillOnClosed,ServiceType,ServiceLine
P-1311,3011 - Data,3011,Closed,null,null,2018-04-26T00:00:00.000+0000,null,Data,1000.00,250.00,1,0,1,null,null,null,null,null,null,null,Recovery Audit,Profit Recovery
P-1313,3011 - Returns,3011,Closed,null,null,2018-04-26T00:00:00.000+0000,null,Returns,1000.00,250.00,1,0,1,null,null,null,null,null,null,null,Recovery Audit,Profit Recovery
P-1314,3010 - Data,3010,Closed,null,null,2018-04-26T00:00:00.000+0000,null,Data,1000.00,250.00,1,0,1,null,null,null,null,null,null,null,Recovery Audit,Profit Recovery
P-1315,3010 - Dupes,3010,Closed,null,null,2018-04-26T00:00:00.000+0000,2018-12-06T00:00:00.000+0000,Dupes,1000.00,250.00,1,0,1,null,null,null,null,null,null,null,Recovery Audit,Profit Recovery
P-1317,3008 - Data,3008,Closed,null,null,2018-04-26T00:00:00.000+0000,2018-05-31T00:00:00.000+0000,Data,1000.00,250.00,1,0,1,null,null,null,null,null,null,null,Recovery Audit,Profit Recovery
P-1318,3008 - Dupes,3008,Closed,null,null,2018-04-26T00:00:00.000+0000,2018-05-31T00:00:00.000+0000,Dupes,1000.00,250.00,1,0,1,null,null,null,null,null,null,null,Recovery Audit,Profit Recovery
P-1319,3008 - Returns,3008,Closed,null,null,2018-04-26T00:00:00.000+0000,2018-05-31T00:00:00.000+0000,Returns,1000.00,250.00,1,0,1,null,null,null,null,null,null,null,Recovery Audit,Profit Recovery
P-1320,3007 - Data,3007,Closed,null,null,2018-04-26T00:00:00.000+0000,null,Data,1000.00,250.00,1,0,1,null,null,null,null,null,null,null,Recovery Audit,Profit Recovery
P-1321,3007 - Dupes,3007,Closed,null,null,2018-04-26T00:00:00.000+0000,null,Dupes,1000.00,250.00,1,0,1,null,null,null,null,null,null,null,Recovery Audit,Profit Recovery
P-1322,3007 - Returns,3007,Closed,null,null,2018-09-19T00:00:00.000+0000,null,Returns,1000.00,250.00,1,0,1,null,null,null,null,null,null,null,Recovery Audit,Profit Recovery


In [0]:
# dbutils.fs.cp('/tmp/requests', '/mnt/smlake/bronze/requests', recurse=True)

Out[50]: True